## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

# **Data Binning, Cleaning, and Standardizing**

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

**'APPLICATION_TYPE' Binning**

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts_df = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts_df = app_type_counts_df.reset_index()
app_type_counts_df.columns = ['APPLICATION_TYPE', 'COUNT']
print(app_type_counts_df)

   APPLICATION_TYPE  COUNT
0                T3  27037
1                T4   1542
2                T6   1216
3                T5   1173
4               T19   1065
5                T8    737
6                T7    725
7               T10    528
8                T9    156
9               T13     66
10              T12     27
11               T2     16
12              T25      3
13              T14      3
14              T29      2
15              T15      2
16              T17      1


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = []

for index, row in app_type_counts_df.iterrows():
  if row['COUNT'] < 500:
    application_types_to_replace.append(row['APPLICATION_TYPE'])

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# # Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

**'CLASSIFICATION' Cleaning**

In [6]:
# Look at CLASSIFICATION value counts for binning
class_counts_df = application_df['CLASSIFICATION'].value_counts()
class_counts_df = class_counts_df.reset_index()
class_counts_df.columns = ['CLASSIFICATION', 'COUNT']
print(class_counts_df)

   CLASSIFICATION  COUNT
0           C1000  17326
1           C2000   6074
2           C1200   4837
3           C3000   1918
4           C2100   1883
..            ...    ...
66          C4120      1
67          C8210      1
68          C2561      1
69          C4500      1
70          C2150      1

[71 rows x 2 columns]


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_to_keep = []

for index, row in class_counts_df.iterrows():
  if row['COUNT'] > 1:
    class_to_keep.append(row['CLASSIFICATION'])

application_df = application_df[application_df['CLASSIFICATION'].isin(class_to_keep)]

application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

for index, row in class_counts_df.iterrows():
  if row['COUNT'] < 1000:
    classifications_to_replace.append(row['CLASSIFICATION'])

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
Name: count, dtype: int64

**Standardizing Data**

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_std_df = pd.get_dummies(application_df, columns=['INCOME_AMT','APPLICATION_TYPE',
                                                             'CLASSIFICATION',
                                                             'SPECIAL_CONSIDERATIONS', 'AFFILIATION', 'STATUS', 'USE_CASE', 'ORGANIZATION'])
application_std_df.head()

,ASK_AMT,IS_SUCCESSFUL,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,...,STATUS_1,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,5000,1,True,False,False,False,False,False,False,False,...,True,False,False,False,False,True,True,False,False,False
1,108590,1,False,True,False,False,False,False,False,False,...,True,False,False,False,True,False,False,True,False,False
2,5000,0,True,False,False,False,False,False,False,False,...,True,False,False,False,False,True,True,False,False,False
3,6692,1,False,False,True,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,True
4,142590,1,False,False,False,True,False,False,False,False,...,True,False,True,False,False,False,False,False,False,True


# **Model Creation**

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_std_df.IS_SUCCESSFUL.values
X = application_std_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3600      
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 4531 (17.70 KB)
Trainable params: 4531 (17.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.5758 - accuracy: 0.7155
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5535 - accuracy: 0.7327
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7326
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7342
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7357
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7349
Epoch 7/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5446 - accuracy: 0.7357
Epoch 8/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5450 - accuracy: 0.7351
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5438 - accuracy: 0.7360
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5438 - accuracy: 0.7372

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5621 - accuracy: 0.7217 - 560ms/epoch - 2ms/step
Loss: 0.562143087387085, Accuracy: 0.7216711640357971


In [16]:
# Export our model to HDF5 file
nn.save("./AlphabetSoupCharity.h5")

from google.colab import files
files.download('./AlphabetSoupCharity.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>